# MR image preprocessing and Radiomic features extraction 
## Image preprocessing
MR images will be preprocessed with bias field correction, noise filtering and intensity normalized.
> Bias field correction is performed with [N4 alorithm](https://simpleitk.readthedocs.io/en/master/link_N4BiasFieldCorrection_docs.html). Simple ITK must be installed: `pip install SimpleITK`

> Noise filtering is performed with gradient anisotropic filter, using [medpy code](https://loli.github.io/medpy/generated/medpy.filter.smoothing.anisotropic_diffusion.html#medpy.filter.smoothing.anisotropic_diffusion). Medpy must be installed: `pip3 install medpy`

> Intensity normalization is performed with Nyul algorithm, code from [intensity-normalization](https://github.com/jcreinhold/intensity-normalization). Installation of the package: `pip install intensity-normalization`

Radiomic feature extraction is performed with pyradiomics

In [1]:
#!/usr/bin/env python

from __future__ import print_function

import collections
import csv
import logging
import os

import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import itk

#import radiomics
#from radiomics import featureextractor

import nibabel as nib

## Read Data
Data are saved as nifti files. Folder path and names of data and masks are stored in a csv file, in current folder.

In [1]:
import os
import csv

out_path = r''
input_csv = os.path.join(out_path, 'DataToCompute.csv')  # !!! change csv filename if needed !!!

f_lists = []
try:
    with open(input_csv, 'r') as in_file:
        csv_reader = csv.DictReader(in_file, lineterminator='\n')
        f_lists = [row for row in csv_reader]
except Exception:
    print("CSV READ FAILED")

image_filepath = []
image_id = []
mask_filepath = []
mask_temp = []
for entry in f_lists:
    image_filepath.append(entry['Image'])
    image_id.append(entry['ID'])
    num_mask = len(entry) - 2
    k = 0

    while k < num_mask:
        k += 1
        mask_name = 'Mask' + str(k)
        mask_temp.append(entry[mask_name])
    mask_filepath.append(",".join(mask_temp))
    mask_temp.clear()


In [2]:
print(mask_filepath)

['Data/Patient63/mask_dir/Patient63_IRM_simu_mridian_gtv.nii,Data/Patient63/mask_dir/Patient63_IRM_simu_mridian_gtv.nii', 'Data/Patient63/mask_dir/Patient63_mridian_ttt_1_gtv.nii,Data/Patient63/mask_dir/Patient63_mridian_ttt_1_gtv.nii']


## N4 bias field correction

In [ ]:

print("N4 bias correction runs.")

bf_repert = 'Data/BFcorr'
if not os.path.exists(bf_repert):
    os.makedirs(bf_repert)

mask_path = 'Data/Masks_WholePat'
if not os.path.exists(mask_path):
    os.makedirs(mask_path)

list_whole_mask = []
list_nam_bfc = []
for name_im_or, nam_id in zip(image_filepath, image_id):

    input_image = sitk.ReadImage(name_im_or)
    mask_image = sitk.OtsuThreshold(input_image, 0, 1, 50)

    input_image = sitk.Cast(input_image, sitk.sitkFloat32)

    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    img_bf_corr = corrector.Execute(input_image, mask_image)

    name_bf_corr = os.path.join(bf_repert, nam_id + '_BFc.nii')
    sitk.WriteImage(img_bf_corr, name_bf_corr)
    list_nam_bfc.append(name_bf_corr)

    mask_wp = os.path.join(mask_path, nam_id + '_mask.nii')
    sitk.WriteImage(mask_image, mask_wp)
    list_whole_mask.append(mask_wp)

print("Finished N4 Bias Field Correction.....")



N4 bias correction runs.
Finished N4 Bias Field Correction.....


## Noise filtering
Settings:
> Number of iteration: 5

> kappa: 5

> gamma: 3

In [4]:
from medpy.io import load, save, get_pixel_spacing
from medpy.filter.smoothing import anisotropic_diffusion

print("Noise filtering runs.")

nfRepert = 'Data/NFcorr'
if not os.path.exists(nfRepert):
 os.makedirs(nfRepert)

# Noise filtering settings
numb_it = 5
kappa = 5
gamma = 3

listNamNfc = []
for nameBFc,namID in zip(listNamBfc,imageID):
    data_input, header_input = load(nameBFc)
    data_output = anisotropic_diffusion(data_input, numb_it, kappa, gamma, get_pixel_spacing(header_input))
    data_output[data_output<=0] = 0
    
    nameNfCorr = nfRepert + '/' + namID + '_NFc.nii'
    listNamNfc.append(nameNfCorr)
    save(data_output, nameNfCorr, header_input)


    
print("Finished Noise filtering.....")

Noise filtering runs.
Finished Noise filtering.....


## Radiomics extraction
A setting file in .yalm is uploaded and contains all the needed settings for features extraction (feature class to extract, number of bins or binwidth, interpolation, resample parameters...)

Feature values are stored in a csv file (outputFilepath)

In [5]:
print("Feature extractation.")

params = os.path.join(outPath, 'Params_MRIdian.yaml')
outputFilepath = os.path.join(outPath, 'extractedFeature.csv')
# Initialize feature extractor using the settings file
extractor = featureextractor.RadiomicsFeatureExtractor(params)
headers = None

datasetToExtract = {}
for idExtr, imgExtr, mskExtr in zip(imageID, listNamNfc, maskFilepath):
    allMasks = mskExtr.split(',') 
    
    for eachMask in allMasks:
        datasetToExtract['ID'] = idExtr
        datasetToExtract['Image'] = imgExtr
        datasetToExtract['Mask'] = eachMask
        
        try:
            datasetToExtract.update(extractor.execute(imgExtr, eachMask))
            
            with open(outputFilepath, 'a') as outputFile:
                writer = csv.writer(outputFile, lineterminator='\n')
                if headers is None:
                    headers = list(datasetToExtract.keys())
                    writer.writerow(headers)
                row = []
                for h in headers:
                    row.append(datasetToExtract.get(h, "N/A"))
                writer.writerow(row)
            
        except Exception:
            print('FEATURE EXTRACTION FAILED')
            
        
print("...End....")        

Feature extractation.


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


...End....
